# Byte pair encoding (BPE)

Byte Pair Encoding (BPE) is a data compression algorithm that learns to merge
frequent pairs of tokens into single tokens.

How it works:
1. Start with individual bytes (or characters) as tokens
2. Find the most frequent pair of consecutive tokens
3. Merge that pair into a new single token
4. Repeat steps 2-3 until desired vocabulary size is reached

Example:
Starting with text: "hello hello"
- Initial tokens: ['h', 'e', 'l', 'l', 'o', ' ', 'h', 'e', 'l', 'l', 'o']
- Most frequent pair: ('l', 'l') appears 2 times
- Merge 'll' into a single token: ['h', 'e', 'll', 'o', ' ', 'h', 'e',
  'll', 'o']
- Next most frequent pair: ('h', 'e') appears 2 times
- Merge 'he' into a single token: ['he', 'll', 'o', ' ', 'he', 'll', 'o']
- Continue until vocabulary size is reached

BPE is commonly used in modern language models (like GPT) to create
efficient tokenizers that can handle any text while maintaining a reasonable
vocabulary size.

In [1]:
text = (
    "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear "
    "and awe into the hearts of programmers worldwide. We all know we ought "
    "to "
    '"support Unicode" in our software (whatever that means—like using '
    "wchar_t for all the strings, right?). But Unicode can be abstruse, and "
    "diving into the thousand-page Unicode Standard plus its dozens of "
    "supplementary annexes, reports, and notes can be more than a little "
    "intimidating. I don't blame programmers for still finding the whole "
    "thing mysterious, even 30 years after Unicode's inception."
)

tokens_bytes = text.encode("utf-8")  # raw bytes
tokens = list(
    tokens_bytes
)  # convert to a list of integers in range 0..255 for convenience
print("---")
print(text)
print("length: ", len(text))
print("---")
print(tokens)
print("length:", len(tokens))

---
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to "support Unicode" in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don't blame programmers for still finding the whole thing mysterious, even 30 years after Unicode's inception.
length:  533
---
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 14

In [2]:
def get_stats(ids):
    """Count the frequency of consecutive pairs in a list of IDs.

    Args:
        ids: A list of integers representing token IDs.

    Returns:
        A dictionary mapping pairs (tuples) to their occurrence counts.

    Example:
        >>> get_stats([1, 2, 2, 3, 2, 2])
        {(1, 2): 1, (2, 2): 2, (2, 3): 1, (3, 2): 1}
    """
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts


stats = get_stats(tokens)
value_key_list = [(v, k) for k, v in stats.items()]
print(sorted(value_key_list, reverse=True))

[(20, (101, 32)), (15, (240, 159)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (226, 128)), (8, (116, 104)), (7, (159, 135)), (7, (159, 133)), (7, (97, 114)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (116, 32)), (6, (114, 32)), (6, (111, 114)), (6, (110, 103)), (6, (110, 100)), (6, (109, 101)), (6, (104, 101)), (6, (101, 114)), (6, (32, 105)), (5, (117, 115)), (5, (115, 116)), (5, (110, 32)), (5, (100, 101)), (5, (44, 32)), (5, (32, 115)), (4, (116, 105)), (4, (116, 101)), (4, (115, 44)), (4, (114, 105)), (4, (111, 117)), (4, (111, 100)), (4, (110, 116)), (4, (110, 105)), (4, (105, 99)), (4, (104, 97)), (4, (103, 32)), (4, (101, 97)), (4, (100, 32)), (4, (99, 111)), (4, (97, 109)), (4, (85, 110)), (4, (32, 119)), (4, (32, 111)), (4, (32, 102)), (4, (32, 85)), (3, (118, 101)), (3, (116, 115)), (3, (116, 114)), (3, (116, 111)), (3, (114, 116)), (3, (114, 115)), (3, (114, 101)), (3, (111, 102)), (3, (111, 32)), (3, (108, 108)), (3

In [3]:
top_pair = max(stats, key=lambda x: stats[x])
print(top_pair)

(101, 32)


In [4]:
def merge(ids, pair, idx):
    """Replace all occurrences of a consecutive pair with a new token ID.

    Args:
        ids: A list of token IDs.
        pair: A tuple of two consecutive values to find and replace.
        idx: The new token ID to replace the pair with.

    Returns:
        A new list with all occurrences of the pair replaced by idx.

    Example:
        >>> merge([5, 6, 6, 7, 9, 1], (6, 7), 99)
        [5, 6, 99, 9, 1]
    """
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i + 1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids


print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))  # [5, 6, 99, 9, 

[5, 6, 99, 9, 1]


In [5]:
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print("length:", len(tokens2))


[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 256, 118, 101, 114, 121, 32, 110, 97, 109, 256, 115, 116, 114, 105, 107, 101, 115, 32, 102, 101, 97, 114, 32, 97, 110, 100, 32, 97, 119, 256, 105, 110, 116, 111, 32, 116, 104, 256, 104, 101, 97, 114, 116, 115, 32, 111, 102, 32, 112, 114, 111, 103, 114, 97, 109, 109, 101, 114, 115, 32, 119, 111, 114, 108, 100, 119, 105, 100, 101, 46, 32, 87, 256, 97, 108, 108, 32, 107, 110, 111, 119, 32, 119, 256, 111, 117, 103, 104, 116, 32, 116, 111, 32, 34, 115, 117,

In [6]:
VOCAB_SIZE = 276  # desired final vocabulary size
NUM_MERGES = VOCAB_SIZE - 256
ids = list(tokens)

merges = {}  # (int, int) -> int
for i in range(NUM_MERGES):
    stats = get_stats(ids)
    pair = max(stats.items(), key=lambda x: x[1])[0]
    idx = 256 + i
    print(f"merge {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx


merge (101, 32) into a new token 256
merge (240, 159) into a new token 257
merge (105, 110) into a new token 258
merge (115, 32) into a new token 259
merge (97, 110) into a new token 260
merge (226, 128) into a new token 261
merge (116, 104) into a new token 262
merge (257, 133) into a new token 263
merge (257, 135) into a new token 264
merge (97, 114) into a new token 265
merge (239, 189) into a new token 266
merge (261, 140) into a new token 267
merge (267, 264) into a new token 268
merge (101, 114) into a new token 269
merge (111, 114) into a new token 270
merge (116, 32) into a new token 271
merge (258, 103) into a new token 272
merge (115, 116) into a new token 273
merge (260, 100) into a new token 274
merge (32, 262) into a new token 275
